In [ ]:
import tensorflow as tf
import numpy as np

class LearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, decay_rate):
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate

    def __call__(self, step):
        return self.initial_learning_rate * (self.decay_rate ** (step / self.decay_steps))

class CharacteristicLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(CharacteristicLayer, self).__init__()
        self.snn = self.add_weight(shape=(), initializer=tf.random_uniform_initializer(minval=-1, maxval=1), trainable=True)

    def call(self, inputs):
        t = inputs[..., 0]
        indices = inputs[..., 1]
        t = tf.cast(t, tf.float32)
        indices = tf.cast(indices, tf.float32)
        return indices - self.snn * t

class FpuWaveNet(tf.keras.Model):
    def __init__(self):
        super(FpuWaveNet, self).__init__()
        self.characteristic_layer = CharacteristicLayer()
        self.q_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation='tanh', kernel_initializer='lecun_normal'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(512, activation='tanh', kernel_initializer='lecun_normal'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1)
        ])
        self.p_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation='tanh', kernel_initializer='lecun_normal'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(512, activation='tanh', kernel_initializer='lecun_normal'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        z = self.characteristic_layer(inputs)
        z = tf.expand_dims(z, -1)
        q = self.q_layers(z)
        p = self.p_layers(z)
        return q, p

def loss_function(model, t_points, indices, alpha):
    t_grid, idx_grid = tf.meshgrid(t_points, indices, indexing='ij')
    inputs = tf.stack([t_grid, idx_grid], axis=-1)
    q, p = model(inputs)
    N = indices.shape[0] // 2

    q_m1 = tf.roll(q, shift=1, axis=1)  # q_{m-1}
    q_p1 = tf.roll(q, shift=-1, axis=1)  # q_{m+1}
    dp_dt = q_p1 + q_m1 - 2 * q + alpha * (tf.pow(q_p1 - q, 2) - tf.pow(q - q_m1, 2))
    dq_dt = p

    # Calculate component loss functions
    residual_q = tf.reduce_mean(tf.square(dq_dt - p), axis=None)
    residual_p = tf.reduce_mean(tf.square(dp_dt - tf.roll(p, shift=-1, axis=1)), axis=None)
    Loss_GE = residual_q + residual_p

    Loss_BC = (tf.square(q[0, -N + 1] - q[0, -N]) +
               tf.square(q[0, N] - q[0, N - 1]) +
               tf.square(p[0, -N + 1] - p[0, -N]) +
               tf.square(p[0, N] - p[0, N - 1]))

    Loss_Limit = (tf.square(q[0, -N]) + tf.square(q[0, N]) +
                  tf.square(p[0, -N]) + tf.square(p[0, N]))

    Loss_Trans = tf.square(q[0, 0])

    return Loss_GE + Loss_BC + Loss_Limit + Loss_Trans

def train(model, epochs, N, t_range, alpha):
    indices = tf.range(-N, N + 1, dtype=tf.float32)

    # Seperate learning rates for network variables and wave speed
    lr_schedule_s = LearningRateSchedule(0.00001, 1000, 0.9)  # Decreased learning rate for wave speed
    lr_schedule_qp = LearningRateSchedule(0.00001, 1000, 0.9)  # Increased learning rate for network weights
    optimizer_s = tf.keras.optimizers.Adam(learning_rate=lr_schedule_s)
    optimizer_qp = tf.keras.optimizers.Adam(learning_rate=lr_schedule_qp)

    # List to store losses for plotting
    loss_history = []
    speed_history = []

    for epoch in range(epochs):
        # Randomly sample t_points within each epoch
        t_points = tf.random.uniform(shape=(500,), minval=t_range[0], maxval=t_range[1], dtype=tf.float32)

        with tf.GradientTape() as tape:
            loss = loss_function(model, t_points, indices, alpha)
        gradients = tape.gradient(loss, model.trainable_variables)

        # Apply gradient clipping
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

        # Apply different optimizers to different variables
        # Wave speed
        grad_s = [grad for grad, var in zip(gradients, model.trainable_variables) if 'characteristic_layer' in var.name]
        var_s = [var for var in model.trainable_variables if 'characteristic_layer' in var.name]
        optimizer_s.apply_gradients(zip(grad_s, var_s))
        # Network weights
        grad_qp = [grad for grad, var in zip(gradients, model.trainable_variables) if 'characteristic_layer' not in var.name]
        var_qp = [var for var in model.trainable_variables if 'characteristic_layer' not in var.name]
        optimizer_qp.apply_gradients(zip(grad_qp, var_qp))

        # Record loss and predicted every 10 epochs (print every 100)
        if epoch % 10 == 0:
            loss_value = loss.numpy()
            loss_history.append(loss_value)
            speed_history.append(model.characteristic_layer.snn.numpy())
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss_value}')

    return loss_history, speed_history

# Model training
model = FpuWaveNet()
N = 32  # Number of particles on each side of the lattice
alpha = 0.1  # Non-linear coefficient
t_range = (-200, 200)  # Simulation time range
n_epochs = 1000

loss_history, speed_history = train(model, n_epochs, N, t_range, alpha)

Epoch 0, Loss: [0.7492722]
Epoch 100, Loss: [0.02228179]
Epoch 200, Loss: [0.00220394]
Epoch 300, Loss: [0.00107712]
Epoch 400, Loss: [0.00020846]
Epoch 500, Loss: [0.0003935]
Epoch 600, Loss: [0.00153131]
Epoch 700, Loss: [0.00019624]
Epoch 800, Loss: [0.0009381]


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tensorflow as tf

# Constants
a = 10  # Spatial domain endpoint
time_duration = 100  # Time for simulation in arbitrary units
time_steps = 100  # Number of time steps in the animation

# Generate space and time points for prediction
indices = np.linspace(-N, N, 2 * N + 1)  # particle indices, discretized model
time_values = np.linspace(0, time_duration, time_steps)  # Time domain for the animation

# Prepare input for predictions
test_inputs = np.array([[t, idx] for t in time_values for idx in indices])

# Predict using the trained model
q_predictions, p_predictions = model(test_inputs.astype(np.float32))
q_predictions = q_predictions.numpy().reshape((time_steps, -1))  # Reshape q predictions
p_predictions = p_predictions.numpy().reshape((time_steps, -1))  # Reshape p predictions

# Create the animation for 'q' using dots
fig, ax = plt.subplots()
line, = ax.plot(indices, q_predictions[0, :], 'o', color='b', label='q(t)', lw=2)  # Just dots
# line, = ax.plot(indices, q_predictions[0, :], '-o', color='b', label='q(t)', lw=2) # Interpolation

def update(frame):
    # Update the data with dots
    line.set_ydata(q_predictions[frame, :])
    return line,

ani = FuncAnimation(fig, update, frames=time_steps, blit=True)
ax.set_xlim([indices.min(), indices.max()])
ax.set_ylim([q_predictions.min() * 1.1, q_predictions.max() * 1.1])
ax.set_xlabel('Particle Index')
ax.set_ylabel('q(t)')
ax.set_title('Discrete Traveling Wave Solution Over Time')

# Save the animation
ani.save('discrete_traveling_wave.mp4', writer='ffmpeg', fps=15)

plt.show()


NameError: name 'N' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import tensorflow as tf

# Assuming model is defined and loaded correctly elsewhere
# Constants
a = 10  # Spatial domain endpoint
N = 10  # Number of particles
time_duration = 100  # Time for simulation in arbitrary units
time_steps = 100  # Number of time steps in the animation

# Generate space and time points for prediction
indices = np.linspace(-N, N, 2 * N + 1)  # particle indices, discretized model
time_values = np.linspace(0, time_duration, time_steps)  # Time domain for the animation

# Prepare input for predictions
test_inputs = np.array([[t, idx] for t in time_values for idx in indices])

# Predict using the trained model
q_predictions, p_predictions = model(test_inputs.astype(np.float32))
q_predictions = q_predictions.numpy().reshape((time_steps, -1))  # Reshape q predictions
p_predictions = p_predictions.numpy().reshape((time_steps, -1))  # Reshape p predictions

# Create the animation for 'q' with both lines and dots for better visualization
fig, ax = plt.subplots()
line, = ax.plot(indices, q_predictions[0, :], '-o', color='b', label='q(t)', lw=2)

def update(frame):
    # Update the data, maintaining line and dots
    line.set_ydata(q_predictions[frame, :])
    return line,

ani = FuncAnimation(fig, update, frames=time_steps, blit=True)
ax.set_xlim([indices.min(), indices.max()])
ax.set_ylim([q_predictions.min() * 1.1, q_predictions.max() * 1.1])
ax.set_xlabel('Particle Index')
ax.set_ylabel('q(t)')
ax.set_title('Discrete Traveling Wave Solution Over Time')

# Save the animation
ani.save('discrete_traveling_wave_interpolated.mp4', writer='ffmpeg', fps=15)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Assuming model is defined and loaded correctly elsewhere
# Constants
a = 10  # Spatial domain endpoint
N = 10  # Number of particles
time_duration = 1000  # Time for simulation in arbitrary units
time_steps = 100  # Number of time steps for prediction
plot_times = [0, 9, 30, 20]  # Time points to plot

# Generate space and time points for prediction
indices = np.linspace(-N, N, 2 * N + 1)  # particle indices, discretized model
time_values = np.linspace(0, time_duration, time_steps)  # Time domain for prediction

# Prepare input for predictions
test_inputs = np.array([[t, idx] for t in time_values for idx in indices])

# Predict using the trained model
q_predictions, p_predictions = model(test_inputs.astype(np.float32))
q_predictions = q_predictions.numpy().reshape((time_steps, -1))  # Reshape q predictions
p_predictions = p_predictions.numpy().reshape((time_steps, -1))  # Reshape p predictions

# Plot the results at specified time points
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm']
labels = [f't={t}' for t in plot_times]

for i, t in enumerate(plot_times):
    time_index = np.argmin(np.abs(time_values - t))  # Find the closest time index
    ax.plot(indices, q_predictions[time_index, :], '-o', color=colors[i], label=labels[i], lw=2)

ax.set_xlim([indices.min(), indices.max()])
ax.set_ylim([q_predictions.min() * 1.1, q_predictions.max() * 1.1])
ax.set_xlabel('Particle Index')
ax.set_ylabel('q(t)')
ax.set_title('Discrete Traveling Wave Solution at Different Time Points')
ax.legend()
ax.grid(True)

# Save the plot
plt.savefig('test')
plt.show()

In [ ]:
snn_value = model.characteristic_layer.snn.numpy()
print(f'The trained value of snn is: {snn_value}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(0, n_epochs, 10), speed_history, label='Speed Every 10 Epochs')
plt.xlabel('Epoch')
plt.ylabel('Wave Speed')
plt.title('Estimated Wave Speed Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('speed_plot.png')
plt.show()

In [ ]:
# Plotting the loss history after training
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(0, n_epochs, 10), loss_history, label='Loss Every 10 Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('loss_plot.png')
plt.show()